In [2]:
import cv2
import numpy as np
from djitellopy import Tello

In [3]:
# 배터리 체크
def battery_check() : 
  drone = Tello()
  drone.connect()

  power = drone.get_battery()
  if power < 30 : print("배터리 부족", power)
  else : print("배터리", power)
  drone.end()
  return power
battery_check()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'


배터리 60


60

In [4]:
#  -- 초기값 설정 --
# 스타트flag
start_counter = 1
# 프레임 flag - 이때 딥러닝 예측도 실행해야됨
frame_flag = 100
# 임계값 조절
tolerance_x = 5
tolerance_y = 5
# 속도 한계값
slowdown_threshold_x = 15
slowdown_threshold_y = 15
# 속도
drone_speen_x = 10
drone_speen_y = 10
# 화면 조절
set_point_x = 960/2
set_point_y = 720/2
hull = 0
# x,y,w,h 전역변수
# global x, y, w, h
x,y,w,h = 0,0,0,0


# 드론 초기화
drone = Tello()
drone.connect()

# 스트림 연결 오류 
drone.streamon()

# 모델 임포트
# faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# -- 영상 루프 시작 --
while True :
  # takeoff 설정
  if start_counter == 0 :
    drone.takeoff()
    print('takeoff')
    start_counter = 1

  # 프레임 읽어오기
  frame = drone.get_frame_read().frame
  cv2.circle(frame, (int(set_point_x), int(set_point_y)), 12, (255,0,0), 1) # 화면 중간에 원표시
  
  # flalg 설정
  frame_flag -=1
  # print("frame_flag", frame_flag)
  if frame_flag == 0 :
    print("frame_flag 0일때 ")



    # -- 필터처리 --
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21)
    lower = np.array([0,48,80], dtype="uint8")
    upper = np.array([20,255,255], dtype="uint8")
    img_hand = cv2.inRange(img_hsv,lower,upper)
    
    #경계선 찾음
    contours, _ = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # 가장 큰 영역 찾기
    max = 0
    maxcnt = None
    
    # 이건 뭔지 모름
    for cnt in contours :
      area = cv2.contourArea(cnt)
      if(max < area) :
        max = area
        maxcnt = cnt
    
    # 이것도 뭔지 모름 암튼 예외처리
    if maxcnt != np.array([]):
      mask = np.zeros(frame.shape).astype(frame.dtype)
      # 경계선 내부 255로 채우기
      color = [255, 255, 255]
      cv2.fillPoly(mask, [maxcnt], color)
      img_hand = cv2.bitwise_and(frame, mask)

    hull = cv2.convexHull(maxcnt)
    x, y, w, h = cv2.boundingRect(hull)
    print(x,y,w,h)
    frame_flag = 100

  # 도출된 값으로 사각형 그리기
  cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0),3 )
  cv2.circle(frame, (int(x+w/2) , int(y+h/2)), 12, (255,0,0), 1) # 얼굴 중심 표시

  # -- 드론 제어 --
  #  얼굴중심과 화면중심의 차를 계산
  distance_x = x+w/2 - set_point_x
  distance_y = y+h/2 - set_point_y

  up_down_velocity = 0
  right_left_veiocity = 0
  for_back_veiocity = 0

# 드론 좌우 이동
  if distance_x < -tolerance_x:
    print("left move")
    right_left_veiocity = - drone_speen_x
  elif distance_x > tolerance_x:
    print("right move")
    right_left_veiocity = drone_speen_x
  else :
    print("OK")

  # 드론 상하 이동
  if distance_y < -tolerance_y:
    print("up move")
    up_down_velocity = drone_speen_y
  elif distance_y > tolerance_y:
    print("down move")
    up_down_velocity = - drone_speen_y
  else :
    print("OK")

  # 드론 앞뒤 이동
  if w*h < 960*720/2:
    for_back_veiocity = 10
  elif w*h > 960*720/2:
    for_back_veiocity = -10
  elif w*h <= 960*720 :
    for_back_veiocity = 10
  else:
    print("OK")



#  임계치 이상 벗어나면 속도 조정 
  if abs(distance_x) < slowdown_threshold_x:
    right_left_veiocity = int(right_left_veiocity / 2)
  if abs(distance_y) < slowdown_threshold_y:
    up_down_velocity = int(up_down_velocity / 2)

  #드론 움직이기
  drone.send_rc_control(right_left_veiocity, 0, up_down_velocity, 0)

  # 비디오 띄우기
  cv2.imshow("Video", frame)
  # 키 설정
  key = cv2.waitKey(1)
  if key == ord('q'):
    break

cv2.destroyAllWindows()
drone.streamoff()
drone.end()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 -

left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move


[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'

left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move
left move
up move


[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc -10 0 10 0'

...

[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 10 0'
[INFO] tello.py - 470 - S

OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
OK
up move
frame_flag 0일때 


C:\Users\nono\AppData\Local\Temp\ipykernel_25720\3367555404.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if maxcnt != np.array([]):


error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\imgproc\src\convhull.cpp:137: error: (-215:Assertion failed) total >= 0 && (depth == CV_32F || depth == CV_32S) in function 'cv::convexHull'


: 